# Data preprocessing

---


In [54]:
INPUT_PATH = r'..\..\data\processed\For content base\Final_content_base.csv'

OUTPUT_PATH = r'..\..\data\processed\For content base\similarity_matrix_cb.pkl'

# importing


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import ast
import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances
from sklearn.preprocessing import StandardScaler

import joblib

In [3]:
books_df = pd.read_csv(INPUT_PATH)

In [4]:
books_df.head()

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0.000000,1,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,4.928571,14,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,5.000000,3,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,4.272727,11,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0.000000,1,http://images.amazon.com/images/P/0393045218.0...


In [5]:
books_df.sort_values(['Rating_count', 'Avg_rating'], ascending=False)

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
26,0971880107,Wild Animus,Rich Shapero,2004,Too Far,1.019584,2502,http://images.amazon.com/images/P/0971880107.0...
402,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",4.468726,1295,http://images.amazon.com/images/P/0316666343.0...
735,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,4.652322,883,http://images.amazon.com/images/P/0385504209.0...
1086,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,3.448087,732,http://images.amazon.com/images/P/0060928336.0...
516,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,4.334716,723,http://images.amazon.com/images/P/0312195516.0...
...,...,...,...,...,...,...,...,...
264555,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,Golden Guides from St. Martin's Press,0.000000,1,http://images.amazon.com/images/P/1582380805.0...
264557,014002803X,Anti Death League,Kingsley Amis,1975,Viking Press,0.000000,1,http://images.amazon.com/images/P/014002803X.0...
264561,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,0.000000,1,http://images.amazon.com/images/P/006008667X.0...
264562,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,0.000000,1,http://images.amazon.com/images/P/0192126040.0...


In [6]:
books_df['Author'].value_counts()

Agatha Christie        593
William Shakespeare    547
Stephen King           509
Ann M. Martin          420
Francine Pascal        372
                      ... 
Christine Balint         1
Fernan Savater           1
Karen King               1
Diana Joseph             1
Christopher  Biffle      1
Name: Author, Length: 99983, dtype: int64

In [7]:
books_df['Publisher'].value_counts()

Harlequin                                       7523
Silhouette                                      4190
Pocket                                          3859
Ballantine Books                                3755
Bantam Books                                    3632
                                                ... 
Milestone Books                                    1
A. Bonniers fÃ¶rlag                                1
Texas Fish &amp; Game Publishing Co., L.L.C.       1
Kepustakaan Populer Gramedia                       1
Connaught                                          1
Name: Publisher, Length: 16279, dtype: int64

In [8]:
books_df['Publication_year'].value_counts()

2002    17542
1999    17342
2001    17264
2000    17151
1998    15700
1997    14830
2003    14277
1996    13975
1995    13494
1994    11739
1993    10556
1992     9863
1991     9346
1990     8627
1989     7912
1988     7466
1987     6508
1986     5824
2004     5753
1985     5329
1984     4961
1983     4484
1982     4186
1981     3274
1980     2671
1979     2198
1978     2127
1977     1892
1976     1592
1975     1210
1974     1017
1973      919
1972      768
1971      540
1970      458
1969      338
1968      232
1966      182
1967      176
1965      173
1964      148
1960      133
1961      132
1963      132
1962      123
Name: Publication_year, dtype: int64

In [9]:
books_df['Avg_rating'].value_counts()

0.000000     118023
5.000000      14148
8.000000      13360
10.000000     12834
7.000000      10380
              ...  
0.560000          1
3.107843          1
2.847826          1
3.177419          1
7.888889          1
Name: Avg_rating, Length: 3738, dtype: int64

## Since we have a lot of unique values and this will require a lot of calculations when measuring the similarity matrix(i don't have enough RAM to do that)...we will filter the book data and take the authors who have more than 30 books and Avg rating > 5.


In [10]:
authors_count = books_df['Author'].value_counts()

In [24]:
authors_above_30 = authors_count[authors_count > 30].index.to_list()

In [26]:
filtered_books = books_df[books_df['Author'].isin(authors_above_30)]

In [28]:
filtered_books = filtered_books[filtered_books['Avg_rating'] > 5]

---


## Get book details df


In [29]:
filtered_books.head(1)

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
15,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,6.0,1,http://images.amazon.com/images/P/1567407781.0...


In [30]:
books_dettails = filtered_books[['Author', 'Publication_year', 'Publisher']]

In [31]:
books_dettails['Publication_year'] = books_dettails['Publication_year'].astype(
    'O')

C:\Users\xobed\AppData\Local\Temp\ipykernel_4148\3030580928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_dettails['Publication_year'] = books_dettails['Publication_year'].astype('O')


In [32]:
books_dettails.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9015 entries, 15 to 264559
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Author            9015 non-null   object
 1   Publication_year  9015 non-null   object
 2   Publisher         9015 non-null   object
dtypes: object(3)
memory usage: 281.7+ KB


# Make one hot encoding for book dettails


In [33]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

encoded_book_det = encoder.fit_transform(books_dettails)

In [34]:
encoded_book_det

<9015x2230 sparse matrix of type '<class 'numpy.float64'>'
	with 27045 stored elements in Compressed Sparse Row format>

---


# Measure similarity


Since most of the matrices are 0s and 1s, we will use cosine similarity (or distance) as a means of measuring the similarity between the films.


In [36]:
similarity_books = cosine_similarity(encoded_book_det)
similarity_books

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [38]:
filtered_books

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
15,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,6.000000,1,http://images.amazon.com/images/P/1567407781.0...
90,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...,J. R. R. Tolkien,2001,Minotauro,6.562500,16,http://images.amazon.com/images/P/8445071408.0...
91,8445071769,El Senor De Los Anillos: Las DOS Torres (Lord ...,J. R. R. Tolkien,2001,Minotauro,6.000000,10,http://images.amazon.com/images/P/8445071769.0...
92,8445071777,El Senor De Los Anillos: El Retorno Del Rey (T...,J. R. R. Tolkien,2001,Distribooks,6.500000,10,http://images.amazon.com/images/P/8445071777.0...
175,3257203659,Der illustrierte Mann. ErzÃ?Â¤hlungen.,Ray Bradbury,2002,Diogenes,7.666667,3,http://images.amazon.com/images/P/3257203659.0...
...,...,...,...,...,...,...,...,...
264444,0439230209,Raccoons on the Roof (Animal Ark),Ben M. Baglio,2001,Scholastic,7.000000,1,http://images.amazon.com/images/P/0439230209.0...
264479,0060093269,Two of a Kind #30: Making a Splash (Two of a K...,Mary-Kate &amp; Ashley Olsen,2003,HarperEntertainment,9.000000,1,http://images.amazon.com/images/P/0060093269.0...
264530,3404204433,Am Ende des Regenbogens.,Alan Dean Foster,2002,LÃ?Â¼bbe,8.000000,1,http://images.amazon.com/images/P/3404204433.0...
264539,8420614556,Lewis Carroll: A Traves Del Espejo Y Lo Que Al...,Lewis Carroll,1986,Lectorum Pubns (Adult),7.000000,1,http://images.amazon.com/images/P/8420614556.0...


In [44]:
sim_m = pd.DataFrame(
    similarity_books, columns=filtered_books['ISBN'], index=filtered_books['Title'])
sim_m.head()

ISBN,1567407781,8445071408,8445071769,8445071777,3257203659,3257207522,3257208634,3257212429,3257214154,3257216416,...,8484504794,9500416689,0307129683,0395647398,5550534274,0439230209,0060093269,3404204433,8420614556,0440400988
Title,,,,,,,,,,,,,,,,,,,,,
The Witchfinder (Amos Walker Mystery Series),1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
El Senor De Los Anillos: LA Comunidad Del Anillo (Lord of the Rings (Spanish)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
El Senor De Los Anillos: Las DOS Torres (Lord of the Rings (Paperback)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
"El Senor De Los Anillos: El Retorno Del Rey (Tolkien, J. R. R. Lord of the Rings. 3.)",0.0,0.666667,0.666667,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
Der illustrierte Mann. ErzÃ?Â¤hlungen.,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.333333,0.333333,0.333333,0.333333,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0


In [46]:
sim_m['1567407781'].sort_values(ascending=False).head(10)

Title
The Witchfinder (Amos Walker Mystery Series)                                                   1.000000
Jitterbug: A Novel of Detroit                                                                  0.666667
The Witchfinder (Bookcassette(r) Edition)                                                      0.666667
Blood Work (Nova Audiobooks)                                                                   0.666667
The Ninth Garfield Fat Cat 3-Pack (Garfield Fat Cat Three Pack)                                0.333333
Arthur and the Popularity Test : A Marc Brown Arthur Chapter Book 12 (Arthur Chapter Books)    0.333333
For Kicks                                                                                      0.333333
The Transall Saga                                                                              0.333333
How to Study the Bible for Yourself                                                            0.333333
The Picture of Dorian Gray (Oxford World's Classics)      

---


---


### Test recommendation based on similarity matrix


In [49]:
def recommend(book_id):
    return sim_m[book_id].sort_values(ascending=False).head(10)

In [53]:
recommend('0425098451')

Title
A Pocket Full of Rye (Miss Marple Mysteries (Paperback))                 1.000000
N or M?                                                                  1.000000
Cards on the Table (Agatha Christie Mysteries Collection (Paperback))    0.666667
Cards on the Table                                                       0.666667
Murder in Three Acts                                                     0.666667
Evil Under the Sun (Hercule Poirot Mysteries (Paperback))                0.666667
After the Funeral (Hercule Poirot Mysteries (Paperback))                 0.666667
Halloween Party (Hercule Poirot Mysteries (Paperback))                   0.666667
The Regatta Mystery                                                      0.666667
Five Little Pigs (Agatha Christie Mysteries Collection (Paperback))      0.666667
Name: 0425098451, dtype: float64

---


# save similarity matrix


In [56]:
joblib.dump(sim_m, OUTPUT_PATH)

['..\\..\\data\\processed\\For content base\\similarity_matrix_cb.pkl']